#Importando bibliotecas

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping

#Carregar dataset

In [9]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)

# Pré-processamento: redimensionar e normalizar imagens

In [10]:
def format_example(image, label):
    image = tf.image.resize(image, (224, 224))  # Tamanho esperado pela maioria das CNNs pré-treinadas
    image = image / 255.0  # Normalizar para [0, 1]
    return image, label

train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

#Realizar o Transfer Learning com um modelo pré-treinado

In [11]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,  # Remover a camada final (queremos adaptar ao nosso problema)
    weights='imagenet'  # Pesos pré-treinados na ImageNet
)

# Congelar a base (opcional, mas comum em Transfer Learning)
base_model.trainable = False

# Adicionar novas camadas no topo
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')  # 1 neurônio para classificação binária
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │           1,281 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

#Treinamento

In [ ]:
history = model.fit(
    train_generator if 'generator' in locals() else train.batch(32),
    epochs=10,
    validation_data=validation_generator if 'generator' in locals() else validation.batch(32)
)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 48s 71ms/step - accuracy: 0.9344 - loss: 0.1731 - val_accuracy: 0.9824 - val_loss: 0.0517
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 71s 55ms/step - accuracy: 0.9834 - loss: 0.0467 - val_accuracy: 0.9832 - val_loss: 0.0481
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 56ms/step - accuracy: 0.9856 - loss: 0.0406 - val_accuracy: 0.9832 - val_loss: 0.0473
Epoch 4/10
390/582 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.9871 - loss: 0.0376

#Avaliação de dados de teste

In [ ]:
test_loss, test_accuracy = model.evaluate(test.batch(32))
print(f"Acurácia no teste: {test_accuracy*100:.2f}%")